In [1]:
import pandas as pd
import numpy as np
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import sys
from os.path import join
import time
import os

from datetime import datetime as dt
import datetime
import pytz
utc = pytz.utc
from glob import glob

In [2]:
!pwd

/Users/helena/projects/OpenTrafficCount/otc-toolkit/recording


In [64]:
PATH_TO_RECORDINGS = "../evaluate_recordings/data" #recording/name
DATETIME_OF_RECORDING = "2020-07-10-11-31-27-614801"
IMAGE_PATH = join(PATH_TO_RECORDINGS, DATETIME_OF_RECORDING) 
DATA = join(IMAGE_PATH, DATETIME_OF_RECORDING + '_tracker.json')
NUMBER_IMAGES = len(glob(join(IMAGE_PATH, '*.png')))

In [9]:
IMAGE_PATH

'data/2020-07-10-11-31-27-614801'

In [8]:
DATA

'data/2020-07-10-11-31-27-614801/2020-07-10-11-31-27-614801_tracker.json'

In [6]:
NUMBER_IMAGES

100

In [7]:
df = pd.read_json(DATA)

In [10]:
df.head()

,_id,recordingId,frameId,timestamp,objects
0,5f0835653a97be0066b7db4f,5f0835653a97be0066b7db4e,3,2020-07-10 09:31:17.671000+00:00,"[{'id': 0, 'x': 173, 'y': 216, 'w': 19, 'h': 3..."
1,5f0835653a97be0066b7db50,5f0835653a97be0066b7db4e,3,2020-07-10 09:31:17.686000+00:00,"[{'id': 0, 'x': 173, 'y': 216, 'w': 19, 'h': 3..."
2,5f0835663a97be0066b7db51,5f0835653a97be0066b7db4e,7,2020-07-10 09:31:18.049000+00:00,"[{'id': 0, 'x': 164, 'y': 219, 'w': 19, 'h': 3..."
3,5f0835663a97be0066b7db52,5f0835653a97be0066b7db4e,7,2020-07-10 09:31:18.072000+00:00,"[{'id': 0, 'x': 164, 'y': 219, 'w': 19, 'h': 3..."
4,5f0835663a97be0066b7db53,5f0835653a97be0066b7db4e,8,2020-07-10 09:31:18.146000+00:00,"[{'id': 0, 'x': 163, 'y': 219, 'w': 21, 'h': 3..."


In [11]:
df.shape

(1008, 5)

In [12]:
df.columns

Index(['_id', 'recordingId', 'frameId', 'timestamp', 'objects'], dtype='object')

In [13]:
df.dtypes

_id                         object
recordingId                 object
frameId                      int64
timestamp      datetime64[ns, UTC]
objects                     object
dtype: object

## Explore times

In [15]:
ffmpeg_start_time = dt(*tuple(int(x) for x in DATETIME_OF_RECORDING.split("-")), tzinfo=utc)
ffmpeg_start_time = ffmpeg_start_time + datetime.timedelta(hours=-2)

In [16]:
ffmpeg_start_time

datetime.datetime(2020, 7, 10, 9, 31, 27, 614801, tzinfo=<UTC>)

In [17]:
# test
ffmpeg_start_time.microsecond

614801

In [18]:
# total time of JSON data
df["timestamp"].max() - df["timestamp"].min()

Timedelta('0 days 00:00:51.033000')

In [19]:
# JSON file starts 40 seconds earlier that ffmpeg recording and stops later
df["timestamp"] - ffmpeg_start_time

0      -1 days +23:59:50.056199
1      -1 days +23:59:50.071199
2      -1 days +23:59:50.434199
3      -1 days +23:59:50.457199
4      -1 days +23:59:50.531199
                 ...           
1003            00:00:40.863199
1004            00:00:40.967199
1005            00:00:40.971199
1006            00:00:41.089199
1007            00:00:41.083199
Name: timestamp, Length: 1008, dtype: timedelta64[ns]

In [20]:
s = abs(df["timestamp"] - ffmpeg_start_time)
s.idxmin()

194

In [21]:
s.loc[s.idxmin()]

Timedelta('0 days 00:00:00.038199')

In [22]:
df.loc[s.idxmin()]

_id                                     5f08356f3a97be0066b7dc11
recordingId                             5f0835653a97be0066b7db4e
frameId                                                      103
timestamp                       2020-07-10 09:31:27.653000+00:00
objects        [{'id': 1, 'x': 241, 'y': 5, 'w': 20, 'h': 11,...
Name: 194, dtype: object

## Modularize and merge

In [23]:
def find_elem_with_closest_ts(json_df, video_start):
    """Find index of row whose timestamp is closest to the video starting time"""
    s = abs(json_df["timestamp"] - video_start)
    return s.idxmin()

In [24]:
def merge(json_df): 
    image_file_names = [PREFIX + '_' + str(i).zfill(3) + '.png' for i in json_df.index]
    merged = {image_file_names[j]:json_df.loc[j,"objects"] for j in range(len(image_file_names))}
    return merged

In [25]:
starting_index = find_elem_with_closest_ts(df, ffmpeg_start_time)

In [26]:
starting_index

194

In [27]:
df.loc[starting_index:].reset_index()

,index,_id,recordingId,frameId,timestamp,objects
0,194,5f08356f3a97be0066b7dc11,5f0835653a97be0066b7db4e,103,2020-07-10 09:31:27.653000+00:00,"[{'id': 1, 'x': 241, 'y': 5, 'w': 20, 'h': 11,..."
1,195,5f08356f3a97be0066b7dc12,5f0835653a97be0066b7db4e,103,2020-07-10 09:31:27.657000+00:00,"[{'id': 1, 'x': 241, 'y': 5, 'w': 20, 'h': 11,..."
2,196,5f08356f3a97be0066b7dc13,5f0835653a97be0066b7db4e,104,2020-07-10 09:31:27.759000+00:00,"[{'id': 1, 'x': 241, 'y': 5, 'w': 20, 'h': 11,..."
3,197,5f08356f3a97be0066b7dc14,5f0835653a97be0066b7db4e,104,2020-07-10 09:31:27.763000+00:00,"[{'id': 1, 'x': 241, 'y': 5, 'w': 20, 'h': 11,..."
4,198,5f08356f3a97be0066b7dc15,5f0835653a97be0066b7db4e,105,2020-07-10 09:31:27.854000+00:00,"[{'id': 1, 'x': 241, 'y': 5, 'w': 20, 'h': 11,..."
...,...,...,...,...,...,...
809,1003,5f0835983a97be0066b7df3a,5f0835653a97be0066b7db4e,507,2020-07-10 09:32:08.478000+00:00,"[{'id': 2, 'x': 253, 'y': 246, 'w': 20, 'h': 3..."
810,1004,5f0835983a97be0066b7df3b,5f0835653a97be0066b7db4e,508,2020-07-10 09:32:08.582000+00:00,"[{'id': 2, 'x': 253, 'y': 246, 'w': 20, 'h': 3..."
811,1005,5f0835983a97be0066b7df3c,5f0835653a97be0066b7db4e,508,2020-07-10 09:32:08.586000+00:00,"[{'id': 2, 'x': 253, 'y': 246, 'w': 20, 'h': 3..."
812,1006,5f0835983a97be0066b7df3e,5f0835653a97be0066b7db4e,509,2020-07-10 09:32:08.704000+00:00,"[{'id': 2, 'x': 253, 'y': 246, 'w': 19, 'h': 3..."


In [54]:
len(df)

1008

In [34]:
df.drop(["_id","recordingId"], inplace=True, axis=1)

In [53]:
df.objects

0       [{'id': 0, 'x': 173, 'y': 216, 'w': 19, 'h': 3...
1       [{'id': 0, 'x': 173, 'y': 216, 'w': 19, 'h': 3...
2       [{'id': 0, 'x': 164, 'y': 219, 'w': 19, 'h': 3...
3       [{'id': 0, 'x': 164, 'y': 219, 'w': 19, 'h': 3...
4       [{'id': 0, 'x': 163, 'y': 219, 'w': 21, 'h': 3...
                              ...                        
1003    [{'id': 2, 'x': 253, 'y': 246, 'w': 20, 'h': 3...
1004    [{'id': 2, 'x': 253, 'y': 246, 'w': 20, 'h': 3...
1005    [{'id': 2, 'x': 253, 'y': 246, 'w': 20, 'h': 3...
1006    [{'id': 2, 'x': 253, 'y': 246, 'w': 19, 'h': 3...
1007    [{'id': 2, 'x': 253, 'y': 246, 'w': 19, 'h': 3...
Name: objects, Length: 1008, dtype: object

In [73]:
df.objects = df.objects.map(lambda x: tuple(x))

In [78]:
df.loc[0,"objects"]==df.loc[1,"objects"]

True

In [79]:
df.loc[0,"objects"]==df.loc[2,"objects"]

False

In [80]:
df.loc[2,"objects"]==df.loc[3,"objects"]

True

In [88]:
for i in range(0,len(df),2):
    #print(i)
    if df.loc[i,"objects"]!=df.loc[i+1,"objects"]:
        print("not equal: ",i)
        print(len(df.loc[i,"objects"]))
        print(len(df.loc[i+1,"objects"]))

not equal:  8
16
15
not equal:  12
15
14
not equal:  20
15
15
not equal:  24
16
15
not equal:  28
15
15
not equal:  30
15
15
not equal:  34
15
14
not equal:  38
15
14
not equal:  44
14
14
not equal:  46
14
14
not equal:  56
16
16
not equal:  58
16
16
not equal:  62
16
15
not equal:  88
16
15
not equal:  96
16
15
not equal:  98
17
17
not equal:  100
17
16
not equal:  108
17
17
not equal:  110
16
17
not equal:  112
16
16
not equal:  116
16
16
not equal:  118
16
15
not equal:  132
16
16
not equal:  140
16
15
not equal:  150
16
16
not equal:  168
16
15
not equal:  172
17
17
not equal:  174
18
18
not equal:  176
18
18
not equal:  178
19
16
not equal:  180
17
16
not equal:  182
16
15
not equal:  184
15
15
not equal:  186
15
15
not equal:  192
16
16
not equal:  194
16
16
not equal:  196
16
15
not equal:  198
15
15
not equal:  202
15
15
not equal:  210
15
14
not equal:  212
16
16
not equal:  218
16
15
not equal:  220
15
15
not equal:  222
15
15
not equal:  226
16
16
not equal:  236
17
16
not e

In [66]:
df.objects.map(lambda x: x[0]["name"]).value_counts()

traffic light    727
car              180
person           101
Name: objects, dtype: int64

In [71]:
df.groupby("objects").groups

TypeError: unhashable type: 'list'

In [40]:
frames = df.groupby("frameId").aggregate(list)

In [41]:
frames.head()

,timestamp,objects
frameId,,
3,"[2020-07-10 09:31:17.671000+00:00, 2020-07-10 ...","[[{'id': 0, 'x': 173, 'y': 216, 'w': 19, 'h': ..."
7,"[2020-07-10 09:31:18.049000+00:00, 2020-07-10 ...","[[{'id': 0, 'x': 164, 'y': 219, 'w': 19, 'h': ..."
8,"[2020-07-10 09:31:18.146000+00:00, 2020-07-10 ...","[[{'id': 0, 'x': 163, 'y': 219, 'w': 21, 'h': ..."
9,"[2020-07-10 09:31:18.249000+00:00, 2020-07-10 ...","[[{'id': 0, 'x': 157, 'y': 220, 'w': 18, 'h': ..."
10,"[2020-07-10 09:31:18.365000+00:00, 2020-07-10 ...","[[{'id': 0, 'x': 153, 'y': 220, 'w': 17, 'h': ..."


In [42]:
frames.shape

(504, 2)

In [45]:
frames.timestamp = frames.timestamp.map(lambda x: set(x))

In [55]:
frames.timestamp.map(lambda x: len(x)).nunique()

1

In [89]:
frames.timestamp.map(lambda x: len(x)).value_counts()

2    504
Name: timestamp, dtype: int64

In [49]:
frames.loc[3,"timestamp"]

{Timestamp('2020-07-10 09:31:17.671000+0000', tz='UTC'),
 Timestamp('2020-07-10 09:31:17.686000+0000', tz='UTC')}

In [50]:
frames.loc[7,"timestamp"]

{Timestamp('2020-07-10 09:31:18.049000+0000', tz='UTC'),
 Timestamp('2020-07-10 09:31:18.072000+0000', tz='UTC')}

In [51]:
df.timestamp.nunique()

1008

In [52]:
frames.loc[509,"timestamp"]

{Timestamp('2020-07-10 09:32:08.698000+0000', tz='UTC'),
 Timestamp('2020-07-10 09:32:08.704000+0000', tz='UTC')}

In [46]:
frames.head()

,timestamp,objects
frameId,,
3,"{2020-07-10 09:31:17.671000+00:00, 2020-07-10 ...","[[{'id': 0, 'x': 173, 'y': 216, 'w': 19, 'h': ..."
7,"{2020-07-10 09:31:18.049000+00:00, 2020-07-10 ...","[[{'id': 0, 'x': 164, 'y': 219, 'w': 19, 'h': ..."
8,"{2020-07-10 09:31:18.146000+00:00, 2020-07-10 ...","[[{'id': 0, 'x': 163, 'y': 219, 'w': 21, 'h': ..."
9,"{2020-07-10 09:31:18.249000+00:00, 2020-07-10 ...","[[{'id': 0, 'x': 157, 'y': 220, 'w': 18, 'h': ..."
10,"{2020-07-10 09:31:18.370000+00:00, 2020-07-10 ...","[[{'id': 0, 'x': 153, 'y': 220, 'w': 17, 'h': ..."


In [28]:
merged = merge(df.loc[starting_index:].reset_index())

NameError: name 'PREFIX' is not defined

In [ ]:
merged

## Draw some boxes for testing

In [ ]:
# objects found in iamge number 1
#objects_1 = merged['2020-07-02-12-07-24-378789_000.png']
objects_1 = merged['2020-07-23-11-31-26-908767_001.png']
filter_list = ['car','traffic light', 'person']
detects = [item for item in objects_1 if item['name'] in filter_list]

#cars_1 = [item for item in objects_1 if item['name']=='car']
#person_1 = [item for item in objects_1 if item['name']=='person']


In [ ]:
len(detects)

#len(cars_1)

In [ ]:
detects

In [65]:
# Draw Bounding boxes
# Set bounding boxes path to same directory as images. 
path_to_images = os.getcwd()+ "/" + IMAGE_PATH
print(path_to_images)
for file_name in os.listdir(path_to_images):
    if file_name.endswith(".png"):
        print(file_name)
        source_img = Image.open(os.path.join(path_to_images, file_name))
        draw = ImageDraw.Draw(source_img)
        for item in detects:
            x = item["x"]
            y = item["y"]
            w = item["w"]
            h = item["h"]
            name = item["name"]
            draw.rectangle(((x-w/2, y-h/2), (x+w/2, y+h/2)), fill=None, outline="red")
            draw.text((x-w/2, y-h/2), name, font=ImageFont.truetype('Pillow/Tests/fonts/Arial.ttf'), fill="red")
        source_img.save('boxes_'+ file_name.split(".")[0] + '.jpg')

   
                                
#(glob(join(IMAGE_PATH, '*.png')))

/Users/helena/projects/OpenTrafficCount/otc-toolkit/recording/../evaluate_recordings/data/2020-07-10-11-31-27-614801
2020-07-10-11-31-27-614801_076.png


NameError: name 'detects' is not defined

In [ ]:
!ls

In [ ]:
#!rm boxes_2020-06-04-15-40-21-286683_*

In [ ]:
!ls Recordings/

In [ ]:
#Open any image with boxes
Image.open("boxes_.jpg")

##Done
1. add text for name parameter

## pending
2. not only cars, all objects
3. all directories
4. store images with boxes, e.g. boxes_<original_image_name>.png (so "boxes" as a prefix for the image name)
5. throw everything into a python script